In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 데이터 세트
# 학습에 필요한 데이터 샘플을 정제하고 정답을 저장하는 기능을 제공한다.
# 클래스 형태로 제공되며,
# 초기화 메서드(__init__), 호출 메서드(__getitem__), 길이 반환 메서드(__len__)를 재정의하여 활용한다.

# 데이터세트 클래스의 기본형

'''
class Dataset:

    def __init__(self, data, *arg, **kwargs):  # 초기화 메서드 (__init__): 입력된 데이터의 전처리 과정을 수행하는 메서드 (학습에 필요한 형태로 변형)
        self.data = data

    def __get__item(self, index):      # 호출 메서드 (__getitem__) : 학습을 진행할때 index에 해당하는 데이터 샘플을 불러오고 반환하는 메서드 
        return tuple(data[index] for data in data.tensors)      # 초기화 메서드에서 변형,개선된 데이터를 가져오며, 데이터 샘플과 정답을 반환한다.

    def __len__(self):        # 길이 반환 메서드(__len__) : 학습에 사용된 전체 데이터세트의 개수를 반환한다. 
        return self.data[0].size(0)
'''

# 모델 학습을 위해 임의의 데이터세트를 구성할 때 파이토치에서 지원하는 데이터 세트 클래스를 상속받아 사용한다.
# 새로 정의한 데이터세트 클래스는 현재 시스템에 적합한 구조로 데이터를 전처리해 사용한다.

'\nclass Dataset:\n\n    def __init__(self, data, *arg, **kwargs):  # 초기화 메서드 (__init__): 입력된 데이터의 전처리 과정을 수행하는 메서드 (학습에 필요한 형태로 변형)\n        self.data = data\n\n    def __get__item(self, index):      # 호출 메서드 (__getitem__) : 학습을 진행할때 index에 해당하는 데이터 샘플을 불러오고 반환하는 메서드 \n        return tuple(data[index] for data in data.tensors)      # 초기화 메서드에서 변형,개선된 데이터를 가져오며, 데이터 샘플과 정답을 반환한다.\n\n    def __len__(self):        # 길이 반환 메서드(__len__) : 학습에 사용된 전체 데이터세트의 개수를 반환한다. \n        return self.data[0].size(0)\n'

In [3]:
# 데이터 로더
# 데이터 세트에 저장된 데이터를 어떠한 방식으로 불러와 활용할지 정의한다.
# 학습을 좀 더 원활하게 진행할수 있도록 배치크기(batch_size), 데이터 순서 변경(shuffle), 데이터 로드 프로세스 수(num_workers)등의 기능을 제공한다.

# 배치크기 (batch_size)는 학습에 사용되는 데이터의 개수가 너무 많아서 한번의 에폭에서 모든데이터를 메모리에 올릴 수 없을 때 데이터를 나누는 역할을 한다.
# 전체 데이터세트에서 배치 크기 만큼 데이터 샘플을 나누고, 모든 배치를 대상으로 학습을 완료하면 한번의 에폭이 완료된다.
#  ex) 1,000개의 데이터 샘플이 데이터 세트의 전체 길이일 떄, 배치크기를 100으로 할당하면 10번의 배치가 완료될때 1번의 에폭이 진행됬다고 볼수있다.

# 데이터 순서 변경 (shuffle)은 모델이 데이터 간의 관계가 아닌, 데이터 순서로 학습되는것을 방지하고자 수행하는 기능이다.
# 데이터 샘플과 정답의 매핑관계는 변경되지 않으며, 행의 순서를 변경하는 개념이다.

# 데이터 로드 프로세스 수 (nun_workers)는 데이터를 불러올 때 사용할 프로세스의 개수를 의미한다.
# 학습을 제외한 코드에서는 데이터를 불러오는데 시간이 가장 오래 소요되므로, 이를 최소화하고자 데이터 로드에 필요한 프로세스의 수를 늘릴수있다.

# 일반적으로 데이터 세트를 재정의해 가장많이 사용하며, 데이터 로더에서는 주로 배치 크기를 조절해 가면서 현재 학습환경에 맞는 구조로 할당한다.

In [4]:
# 다중 선형 회귀
# 기본 구조 선언
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

train_x = torch.FloatTensor([
    [1,2],[2,3],[3,4],[4,5],[5,6],[6,7]
])
train_y = torch.FloatTensor([
    [0.1,1.5],[1,2.8],[1.9,4.1],[2.8,5.4],[3.7,6.7],[4.6,8]
])

# 데이터세트와 데이터로더
train_dataset = TensorDataset(train_x, train_y)
train_dataloader = DataLoader(train_dataset,
                              batch_size=2, # 한번의 배치마다 두개의 데이터 세트와 정답을 가져옴 
                              shuffle=True, # 데이터의 순서를 무작위로 변경
                              drop_last=True) # 배치 크기에 맞지 않는 배치를 제거

# 모델,오차함수.최적화함수 선언
modl = nn.Linear(2, 2, bias=True)  # 다중선형회귀도 선형변환클래스를 사용한다
criterion = nn.MSELoss()
optimizer = optim.SGD(modl.parameters(), lr=0.001)

# 데이터로더 적용
for epoch in range(2000):
    cost = 0.0

    for batch in train_dataloader:
        x, y = batch
        output = model(x)

        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")


NameError: name 'model' is not defined

In [ ]:
# 모델/데이터세트 분리

# 모듈 클래스
class Model(nn.Module):
    def __init__(self):   # 초기화 메서드 (__init__):신경망에 사용될 계층을 초기화한다.
        super().__init__()  # 계층 정의전에 super함수로 모듈 클래스의 속성을 초기화 
        self.conv1 = nn.Conv2d(1,20,5)   #  모델의 매개변수
        self.conv2 = nn.Conv2d(20,20,5)  #  모델의 매개변수

    def forward(self. x): # 순방향 메서드 (forward):모델이 어떤 구조를 갖게 될지를 정의한다.
        x = F.relu(self.conv1(x))    # 데이터(x)를 입력받아 학습을 진행한다
        x = F.relu(self.conv2(x))    # 초기화 메서드에서 super함수로 부모클래스를 초기화 했으므로, backward연산은 정의하지 않아도 됨.
        return x                     # 파이토치의 Autograd에서 모델의 매개변수를 역으로 전파해 자동으로 기울기 or 변화도를 계산해 준다.

In [ ]:
# 비선형회귀
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset,DataLoader

# 사용자 정의 데이터세트
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

# 사용자 정의 모델
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2,1)   # 선형변환함수(nn.Linear)의 입력데이터 차원크기(in_features)는 이차다항식 이므로 2를 입력하고,
                                      #                      출력데이터 차원크기(out_features)는 1을 입력한다    
    def forward(self, x):                 
        x = self.layer(x)             # 순방향 메서드에서 학습과정 정의:
        return x                      # 현재 모델 계층이 하나뿐이므로 self.layer변수에 입력 데이터 x를 전달하고 결괏값을 반환한다.

# 사용자 정의 데이터세트와 데이터로더
train_dataset = CustomDataset("../datasets/non_linear.csv") # train_dataset 변수에 CustomDataset 인스턴스를 생성.
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)  # drop_last : 마지막 배치 제거

# GPU 연산 적용
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")


In [ ]:
# 모델 평가
with torch no_grad():  # no_grad 클래스 : 기울기 계산을 비활성화 하는 클래스로, 자동미분 기능을 사용하지않도록 설정하여 메모리 사용량을 줄여 추론에 적합한 상태로 변경.
    model.eval()      # 모델을 평가모드로 변경
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    )to(device)
    outputs = model(inputs)
    print(outputs)

In [ ]:
# 모델 저장
torch.save(
    model,
    "../models/model.pt"
)

torch.save(
    model.state_dict(),
    "../models/model_state_dict.pt"
)